<a href="https://colab.research.google.com/github/kyle1130/README.md/blob/main/4_llm%EC%9C%BC%EB%A1%9C_%ED%95%99%EC%8A%B5_dataset_%EB%A7%8C%EB%93%A4%EA%B8%B0(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리 설치

In [1]:
!pip install bertopic
!pip install -q transformers bertopic timesfm
!pip install -q scikit-learn

# -*- coding: utf-8 -*-
from google.colab import drive

import io
import os
import json

import torch

import numpy as np
import re
from datetime import datetime
from tqdm import tqdm
from collections import defaultdict

from transformers import AutoModel, AutoTokenizer

import pickle

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

Google drive mount

In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/your_project_folder')
torch.cuda.empty_cache()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


json메시지 로딩

In [3]:
def load_data():
    with open('processed-batch-1.json', 'r') as f:
        data = json.load(f)
    # text가 공백인 메시지는 제외
    return [msg for msg in data if msg['text'].strip()]

messages = load_data()

# 모든 메시지에 anchor_group을 미리 None으로 초기화
# 이후 앵커 트래킹 로직에서 적절히 값이 들어가도록 함
for msg in messages:
    msg['anchor_group'] = None

CHECKPOINT_FILE = "cluster_progress.json"  # 수정: 새 이름
PROCESSED_FILE = 'processed_messages.json'

EEVE 모델 로드

In [4]:
model_path = "/content/drive/MyDrive/eeve_model"

class ResourceManager:
    def __init__(self, model):
        self.model = model
    def __enter__(self):
        self.model_gpu = self.model.to('cuda')
        return self.model_gpu
    def __exit__(self, *args):
        self.model_gpu.to('cpu')
        torch.cuda.empty_cache()

class EEVEModel:
    def __init__(self, model_path):
        # 여기서 tokenizer, model 을 미리 로딩
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModel.from_pretrained(model_path).half().to('cpu')

    def generate_embedding(self, text: str) -> np.ndarray:
        # ResourceManager로 GPU 리소스를 할당받아서 임베딩 계산
        with ResourceManager(self.model) as model_gpu:
            inputs = self.tokenizer(text, return_tensors="pt",
                                    truncation=True,
                                    max_length=512).to('cuda')
            with torch.no_grad():
                outputs = model_gpu(**inputs)
            emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        return emb

BERTopic 로드

In [5]:
# 학습된 BERTopic 모델 불러오기 (Google Drive에 있는 trained_topic_model.pkl 사용)
trained_topic_model = BERTopic.load("trained_topic_model.pkl")

# HybridSimilarity에서 사용할 BERTopic 모델 래퍼 클래스
# transform([text], embeddings=[...])를 호출할 수 있도록 그대로 사용합니다.
class LoadedBERTopic:
    def transform(self, texts, embeddings=None):
        # embeddings 인자를 전달하면 trained_topic_model.transform()에 함께 전달합니다.
        # 모델의 transform()은 (topics, topic_dists)를 반환합니다.
        return trained_topic_model.transform(texts, embeddings=embeddings)
class LoadBERTopic:
    def transform(self, texts):
        """
        실제 BERTopic의 transform(texts) 결과로부터
        topic id나 topic 분포를 얻어서 유사도 계산에 사용.
        여기서는 단순히 0.7~0.8 사이 임의 값으로 가정
        """
        # 예시로 topic distribution을 임의 반환
        # 실제로는 bertopic_model.transform([...]) 형태로 topic 확률 벡터를 얻어야 함
        return [None, [np.array([0.7, 0.3]) for _ in texts]]

# BERTopic 내부에서 사용할 임베딩 모델도 동일하게 로딩해둬야 함 (예: sentence-transformers)
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

mention(호출) 검사 함수

In [6]:
def extract_mentions(text):
    """
    '~님' 형태의 호출 패턴을 단순 정규식으로 추출
    """
    found = re.findall(r"([\w-]+)님", text)
    return set(found)

Hybrid Topic Similarity (EEVE+BERTopic)

In [7]:
class HybridSimilarity:
    def __init__(self, eeve_model, bertopic_model, weight_eeve=0.5):
        self.eeve_model = eeve_model
        self.bertopic_model = bertopic_model  # LoadedBERTopic 인스턴스 전달
        self.weight_eeve = weight_eeve

    def cosine_sim(self, v1, v2):
        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-12)

    def similarity(self, text, group_repr):
        # EEVE 임베딩 유사도 계산
        emb_text = self.eeve_model.generate_embedding(text)
        sim_eeve = self.cosine_sim(emb_text, group_repr["embedding"])

        # BERTopic의 transform()으로 토픽 분포 계산
        # embeddings 인자는 numpy array로 shape (1, vector_dim)이어야 함
        emb_array = np.array([emb_text])
        _, topic_dists = self.bertopic_model.transform([text], embeddings=emb_array)
        sim_bertopic = self.cosine_sim(topic_dists[0], group_repr["topic_dist"])

        hybrid_sim = self.weight_eeve * sim_eeve + (1 - self.weight_eeve) * sim_bertopic
        return hybrid_sim

앵커 트래킹 클래스

In [8]:
class AnchorTracker:
    def __init__(self, eeve_path, threshold=0.5, weight_mention=0.8,
                 merge_threshold=7, merge_similarity=0.8, time_threshold=86400):
        self.eeve_model = EEVEModel(eeve_path)
        self.bertopic_model = LoadedBERTopic()  # 실제 BERTopic 모델 불러오기
        self.sim_model = HybridSimilarity(self.eeve_model, self.bertopic_model, weight_eeve=0.5)
        self.groups = {}  # 그룹 정보 저장 (각 그룹은 {'embedding', 'topic_dist', 'tail_sender', 'tail_text', 'messages'})
        self.current_id = 1
        self.threshold = threshold  # (미사용; 아래 자체 기준으로 새 그룹/기존 그룹 선택)
        self.weight_mention = weight_mention  # ← 호출 문구 점수를 조절하는 가중치 (예: 0.8)
        self.time_threshold = time_threshold  # 예: 24시간(86400초)
        self.merge_threshold = merge_threshold  # on-the-fly merge 수행 메시지 수 기준
        self.merge_similarity = merge_similarity  # on-the-fly merge를 위한 주제 유사도 임계치

    def context_mention_score(self, new_text, group_tail_sender):
        # 앞/뒤 5개 단어 내에 group_tail_sender가 정확히 "xxx님" (호출)로 등장하면 4점,
        # 단순히 텍스트에 포함되어 있으면 2점, 없으면 0점으로 평가합니다.
        mention_score = 0
        pattern_explicit = r"\b" + re.escape(group_tail_sender) + r"님\b"
        if re.search(pattern_explicit, new_text):
            mention_score = 4
        elif group_tail_sender in new_text:
            mention_score = 2
        return mention_score

    def assign_group(self, msg_text, msg_sender, anchor_possible=False):
        current_time = datetime.now()
        # EEVE 임베딩 및 BERTopic 토픽 분포 구하기
        new_emb = self.eeve_model.generate_embedding(msg_text)
        emb_array = np.array([new_emb])
        _, new_topic_dists = self.bertopic_model.transform([msg_text], embeddings=emb_array)
        new_topic = new_topic_dists[0]

        # (1) 만약 메시지에 "anchor_possible": true이면 무조건 새 그룹 생성
        if anchor_possible:
            new_id = self.current_id
            self.current_id += 1
            self.groups[new_id] = {
                "embedding": new_emb,
                "topic_dist": new_topic,
                "tail_sender": msg_sender,
                "tail_text": msg_text,
                "messages": [(msg_text, current_time, new_emb)]
            }
            return new_id

        best_gid = None
        best_total_score = -1

        # (2) 기존 그룹들 순회: 단, 24시간 이내인 그룹만 고려
        for gid, info in self.groups.items():
            last_time = max(info["messages"], key=lambda x: x[1])[1]
            if (current_time - last_time).total_seconds() > self.time_threshold:
                continue

            # mention score: 현재 메시지에서 그룹의 tail_sender가 호출되었는지 확인
            group_tail_sender = info["tail_sender"]
            mention_score = self.context_mention_score(msg_text, group_tail_sender)

            # tail score: 기존 그룹의 마지막 메시지 임베딩과 new_emb의 cosine similarity → 최대 4점
            tail_emb = info["messages"][-1][2]
            tail_sim = self.sim_model.cosine_sim(new_emb, tail_emb)
            tail_score = 4 * tail_sim

            # topic score: new message의 topic 분포와 그룹의 topic_dist 유사도 → 최대 2점
            topic_sim = self.sim_model.cosine_sim(new_topic, info["topic_dist"])
            topic_score = 2 * topic_sim

            total_score = mention_score + tail_score + topic_score
            if total_score > best_total_score:
                best_total_score = total_score
                best_gid = gid

        # 만약 최고 점수가 낮으면 새로운 그룹으로 처리 (예: 점수가 5점 미만이면)
        if best_total_score < 5:
            best_gid = None

        if best_gid is not None:
            # 기존 그룹 업데이트 (가중 평균으로 임베딩 및 topic 업데이트)
            self.groups[best_gid]["embedding"] = 0.7 * self.groups[best_gid]["embedding"] + 0.3 * new_emb
            self.groups[best_gid]["topic_dist"] = 0.7 * self.groups[best_gid]["topic_dist"] + 0.3 * new_topic
            self.groups[best_gid]["tail_sender"] = msg_sender
            self.groups[best_gid]["tail_text"] = msg_text
            self.groups[best_gid]["messages"].append((msg_text, current_time, new_emb))
            # on-the-fly merge 시도: 만약 해당 그룹 내 메시지 수가 merge_threshold 이상이면 병합 시도
            if len(self.groups[best_gid]["messages"]) >= self.merge_threshold:
                self.maybe_merge_group(best_gid)
            return best_gid
        else:
            # 새 그룹 생성
            new_id = self.current_id
            self.current_id += 1
            self.groups[new_id] = {
                "embedding": new_emb,
                "topic_dist": new_topic,
                "tail_sender": msg_sender,
                "tail_text": msg_text,
                "messages": [(msg_text, current_time, new_emb)]
            }
            return new_id

    def maybe_merge_group(self, current_gid):
        """
        on-the-fly merge: 현재 그룹(current_gid)의 메시지 수가 merge_threshold 이상이면,
        기존 그룹들 중 주제 유사도가 merge_similarity 이상인 그룹을 찾아 병합.
        병합 시 임베딩과 topic_dist는 가중 평균으로 업데이트하고 tail 정보 재갱신.
        """
        current_group = self.groups[current_gid]
        for gid, other_group in list(self.groups.items()):
            if gid == current_gid:
                continue
            topic_sim = np.dot(current_group["topic_dist"], other_group["topic_dist"]) / (
                        np.linalg.norm(current_group["topic_dist"]) * np.linalg.norm(other_group["topic_dist"]) + 1e-12)
            if topic_sim >= self.merge_similarity:
                current_group["messages"].extend(other_group["messages"])
                n1 = len(current_group["messages"])
                n2 = len(other_group["messages"])
                total = n1 + n2
                current_group["embedding"] = (n1 * current_group["embedding"] + n2 * other_group["embedding"]) / total
                current_group["topic_dist"] = (n1 * current_group["topic_dist"] + n2 * other_group["topic_dist"]) / total
                latest_msg = max(current_group["messages"], key=lambda x: x[1])
                current_group["tail_text"] = latest_msg[0]
                del self.groups[gid]
                self.maybe_merge_group(current_gid)

# 후처리 merge 함수는 이전 코드와 같이 매우 엄격한 기준(topic_threshold=0.95)을 사용
def merge_anchor_groups(anchor_groups, context_threshold=3600, topic_threshold=0.95):
    """
    post-tracking merge: 24시간 내 tail message들 중, 엄격한 토픽 유사도(topic_threshold=0.95 이상)일 때만 그룹 병합.
    """
    group_list = list(anchor_groups.items())
    merged = {}
    for gid, info in group_list:
        if gid in merged:
            continue
        merged[gid] = info
        tail_msg = sorted(info["messages"], key=lambda x: x[1])[-1]
        for other_gid, other_info in group_list:
            if other_gid == gid or (other_gid in merged and merged[other_gid] is None):
                continue
            other_tail = sorted(other_info["messages"], key=lambda x: x[1])[-1]
            time_diff = abs(tail_msg[1] - other_tail[1]).total_seconds()
            if time_diff > context_threshold:
                continue
            sim = np.dot(info["topic_dist"], other_info["topic_dist"]) / (
                     np.linalg.norm(info["topic_dist"]) * np.linalg.norm(other_info["topic_dist"]) + 1e-12)
            if sim >= topic_threshold:
                merged[gid]["messages"].extend(other_info["messages"])
                updated_emb = 0.5 * (merged[gid]["embedding"] + other_info["embedding"])
                merged[gid]["embedding"] = updated_emb
                merged[other_gid] = None
        merged[gid]["tail_text"] = max(merged[gid]["messages"], key=lambda x: x[1])[0]
    final = {k: v for k, v in merged.items() if v is not None}
    return final


체크포인트 메시지 로딩

In [9]:
if os.path.exists("processed-batch-1.json"):
    with open("processed-batch-1.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    messages = [msg for msg in data if msg["text"].strip()]
else:
    messages = []

for msg in messages:
    if "anchor_group" not in msg:
        msg["anchor_group"] = None

if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, 'r') as f:
        progress = json.load(f)
else:
    progress = {"processed_idx": 0, "failed_ids": []}

# tracker 생성
tracker = AnchorTracker(
    eeve_path="/content/drive/MyDrive/eeve_model",
    threshold=0.5,
    weight_mention=0.8,
    time_threshold=86400,
    merge_threshold=7,
    merge_similarity=0.8  # on-the-fly merge 기준 (post-tracking merge는 별도 함수에서 topic_threshold=0.95 사용)
)

# 디버깅: 초기에 생성된 anchor 그룹 수 출력
print(f"초기 생성된 anchor 그룹 수: {len(tracker.groups)}")


초기 생성된 anchor 그룹 수: 0


임베딩 & 앵커 트래킹

In [10]:
# 메시지 로딩 및 체크포인트 처리 부분은 그대로 사용

# tracker 생성
tracker = AnchorTracker(
    eeve_path="/content/drive/MyDrive/eeve_model",
    threshold=0.5,
    time_threshold=86400,
    merge_threshold=7,
    merge_similarity=0.8  # on-the-fly merge 기준, 후처리 merge는 topic_threshold=0.95 사용
)

print(f"초기 생성된 anchor 그룹 수: {len(tracker.groups)}")

for i in tqdm(range(progress["processed_idx"], len(messages)), desc="임베딩 생성"):
    try:
        if 'embedding' in messages[i] and messages[i].get('anchor_group') not in [None, 0]:
            continue

        emb = tracker.eeve_model.generate_embedding(messages[i]['text']).tolist()
        messages[i]['embedding'] = emb

        sender = messages[i]['sender']
        text   = messages[i]['text']
        anchor_possible = messages[i].get("anchor_possible", False)

        assigned_gid = tracker.assign_group(
            msg_text=text,
            msg_sender=sender,
            anchor_possible=anchor_possible
        )
        messages[i]['anchor_group'] = assigned_gid

        progress["processed_idx"] = i + 1

        if (i + 1) % 10 == 0:
            print(f"메시지 {i+1} 처리 후, 현재 anchor 그룹 수: {len(tracker.groups)}")
            with open(CHECKPOINT_FILE, 'w') as f:
                json.dump(progress, f)
            with open(PROCESSED_FILE, 'w', encoding='utf-8') as f:
                json.dump(messages, f, ensure_ascii=False, indent=4)

    except Exception as e:
        print(f"에러 @ {i}: {str(e)}")
        progress["failed_ids"].append(i)
        continue

with open(CHECKPOINT_FILE, 'w') as f:
    json.dump(progress, f)
with open(PROCESSED_FILE, 'w', encoding='utf-8') as f:
    json.dump(messages, f, ensure_ascii=False, indent=4)

# 후처리 merge (엄격한 기준)
merged_groups = merge_anchor_groups(tracker.groups, context_threshold=3600, topic_threshold=0.95)
print("후처리 merge 후 최종 anchor 그룹 수:", len(merged_groups))


초기 생성된 anchor 그룹 수: 0


임베딩 생성:   4%|▍         | 10/263 [00:12<03:09,  1.34it/s]

메시지 10 처리 후, 현재 anchor 그룹 수: 8


임베딩 생성:   8%|▊         | 20/263 [00:17<01:51,  2.18it/s]

메시지 20 처리 후, 현재 anchor 그룹 수: 12


...

메시지 250 처리 후, 현재 anchor 그룹 수: 57


임베딩 생성:  98%|█████████▊| 259/263 [02:38<00:02,  1.73it/s]

메시지 260 처리 후, 현재 anchor 그룹 수: 61


임베딩 생성: 100%|██████████| 263/263 [02:42<00:00,  1.62it/s]


후처리 merge 후 최종 anchor 그룹 수: 46


anchor group merge

In [11]:
def merge_anchor_groups(anchor_groups, context_threshold=3600, topic_threshold=0.95):
    """
    post-tracking merge: 매우 엄격한 토픽 유사도(topic_threshold=0.95 이상)일 때만 그룹 병합.
    """
    group_list = list(anchor_groups.items())
    merged = {}
    for gid, info in group_list:
        if gid in merged:
            continue
        merged[gid] = info
        tail_msg = sorted(info["messages"], key=lambda x: x[1])[-1]
        for other_gid, other_info in group_list:
            if other_gid == gid or (other_gid in merged and merged[other_gid] is None):
                continue
            other_tail = sorted(other_info["messages"], key=lambda x: x[1])[-1]
            time_diff = abs(tail_msg[1] - other_tail[1]).total_seconds()
            if time_diff > context_threshold:
                continue
            sim = np.dot(info["topic_dist"], other_info["topic_dist"]) / (np.linalg.norm(info["topic_dist"]) * np.linalg.norm(other_info["topic_dist"]) + 1e-12)
            if sim >= topic_threshold:
                merged[gid]["messages"].extend(other_info["messages"])
                updated_emb = 0.5 * (merged[gid]["embedding"] + other_info["embedding"])
                merged[gid]["embedding"] = updated_emb
                merged[other_gid] = None
        # (추가: 엄격한 병합 후 최신 tail 정보 재갱신)
        merged[gid]["tail_text"] = max(merged[gid]["messages"], key=lambda x: x[1])[0]
    final = {k: v for k, v in merged.items() if v is not None}
    return final

# 후처리 merge를 엄격한 기준(topic_threshold=0.95)으로 실행
merged_groups = merge_anchor_groups(tracker.groups, context_threshold=3600, topic_threshold=0.95)
print("후처리 merge 후 최종 anchor 그룹 수:", len(merged_groups))


후처리 merge 후 최종 anchor 그룹 수: 46


최종 결과 저장

In [12]:
group_dict = defaultdict(list)
for msg in messages:
    # 여기서 KeyError가 발생하지 않도록 anchor_group이 None이 아닌지 확인
    if msg['anchor_group'] is None:
        msg['anchor_group'] = -1  # 혹은 기타 임시값
    group_dict[msg['anchor_group']].append(msg['text'])

with open('grouped_messages.json', 'w', encoding='utf-8') as f:
    json.dump(group_dict, f, ensure_ascii=False, indent=4)
